---
### 9.4.1 单因素方差分析方法
#### 1. 模型
$$
\begin{cases}
X_{ij}=\mu+\alpha_i+\varepsilon_{ij},\\
\sum\limits_{i=1}^r n_i\alpha_i=0,\\
\varepsilon_{ij}\sim N(0,\sigma^2),\quad i=1,\cdots,r,\ j=1,\cdots,n_i\
\end{cases}
$$
原假设为
$$
H_0:\alpha_1=\alpha_2=\cdots=\alpha_r=0
$$
#### 2. 统计分析
$$
F=\dfrac{S_A/(r-1)}{S_E/(n-r)}\sim F(r-1, n-r)
$$
检验规则为
$$
F<F_\alpha(r-1, n-r){\text 时接受}H_0{\text ，即认为因素{\mathit A}对试验结果没有显著影响，否则拒绝H_0}
$$
#### 3. 方差分析表
|     方差来源     | 离差平方和 | 自由度 |    均方     |              $F$ 值              | 概率 |
| :--------------: | :--------: | :----: | :---------: | :------------------------------: | :--: |
| 因素 $A$（组间） |   $S_A$    | $r-1$  | $S_A/(r-1)$ | $F=\dfrac{S_A/(r-1)}{S_E/(n-r)}$ | $p$  |
|   误差（组内）   |   $S_E$    | $n-r$  | $S_E/(n-r)$ |                                  |      |
|       总和       |   $S_T$    | $n-1$  |             |                                  |      |

#### 4. Statsmodels 实现
1）

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

df = pd.read_excel('../../09第9章  数据的描述性统计方法/data9_18.xlsx', header=None)
data = df.values.T
y = data[~np.isnan(data)]
x = np.hstack([np.ones(5), np.full(4, 2), np.full(4, 3), np.full(3, 4)])

model_dic = {'x':x, 'y':y}
model = sm.formula.ols('y~C(x)', model_dic).fit()
anovat = sm.stats.anova_lm(model)
anovat

,df,sum_sq,mean_sq,F,PR(>F)
C(x),3.0,60153.333333,20051.111111,3.727742,0.042004
Residual,12.0,64546.666667,5378.888889,NaN,NaN


${\rm PR}=0.042004<0.05$，拒绝 $H_0$，即认为这几种工艺制成的灯泡寿命有显著差异

2）均衡数据

In [2]:
from scipy.stats import f

data = np.loadtxt('../../09第9章  数据的描述性统计方法/data9_19.txt')
mu = data.mean(axis=1)
y = data.flatten()
x = np.tile(np.arange(1, 6), (4, 1)).T.flatten()

model_dic = {'x':x, 'y':y}
model = sm.formula.ols("y~C(x)", model_dic).fit()
anovat = sm.stats.anova_lm(model)
anovat

,df,sum_sq,mean_sq,F,PR(>F)
C(x),4.0,22.7680,5.692000,3.949578,0.02199
Residual,15.0,21.6175,1.441167,NaN,NaN


In [3]:
fa = f.ppf(0.95, anovat.df[0], anovat.df[1])
fa

3.055568275906593

由于 $F=3.9496>3.0556=F_{0.05}(4, 15)$，故拒绝 $H_0$，即认为温度对产品收率有显著影响

### 9.4.2 双因素方差分析方法
#### 1. 数学模型
$$
\begin{cases}
X_{ij}=\mu+\alpha_j+\beta_i+\gamma_{ij}+\varepsilon_{ijk},\\[1ex]
\sum\limits_{j=1}^s\alpha_j=0,\ \sum\limits_{i=1}^r\beta_i=0,\ \sum\limits_{i=1}^r\gamma_{ij}=\sum\limits_{j=1}^s\gamma_{ij}=0,\\[2ex]
\varepsilon_{ijk}\sim N(0,\sigma^2),\quad i=1,\cdots,r,\ j=1,\cdots,s,\ k=1,\cdots,t
\end{cases}
$$
原假设为
$$
\begin{align*}
H_{01}&:\alpha_j=0,\quad j=1,\cdots,s,\\
H_{02}&:\beta_i=0,\quad i=1,\cdots,r,\\
H_{03}&:\gamma_{ij}=0,\quad i=1,\cdots,r,\ j=1,\cdots,s
\end{align*}
$$

#### 2. 无交互影响的双因素方差分析
如果可事先判定两因素之间没有交互影响，每组试验就不必重复，即令 $t=1,\ \gamma_{ij}=0$

当 $H_{01}$ 成立时，
$$
F_A=\dfrac{\dfrac{S_A}{s-1}}{\dfrac{S_E}{(r-1)(s-1)}}\sim F(s-1, (r-1)(s-1))
$$
当 $H_{02}$ 成立时，
$$
F_B=\dfrac{\dfrac{S_B}{r-1}}{\dfrac{S_E}{(r-1)(s-1)}}\sim F(r-1, (r-1)(s-1))
$$
检验规则为
$$
F_A<F_\alpha(s-1, (r-1)(s-1)){\text 时接受{\mathit H_{01}}，否则拒绝{\mathit H_{01}}}\\
F_B<F_\alpha(r-1, (r-1)(s-1)){\text 时接受{\mathit H_{02}}，否则拒绝{\mathit H_{02}}}
$$
双因素方差分析表
| 方差来源 | 离差平方和 |    自由度    |        均方        |                  $F$ 值                   |
| :------: | :--------: | :----------: | :----------------: | :---------------------------------------: |
| 因素 $A$ |   $S_A$    |    $s-1$     |    $S_A/(s-1)$     | $F_A=\dfrac{S_A/(s-1)}{S_E/[(r-1)(s-1)]}$ |
| 因素 $B$ |   $S_B$    |    $r-1$     |    $S_B/(r-1)$     | $F_B=\dfrac{S_B/(r-1)}{S_E/[(r-1)(s-1)]}$ |
|   误差   |   $S_E$    | $(r-1)(s-1)$ | $S_E/[(r-1)(s-1)]$ |                                           |
|   总和   |   $S_T$    |    $rs-1$    |                    |                                           |

In [4]:
data = np.loadtxt('../../09第9章  数据的描述性统计方法/data9_20.txt')
y = data.flatten()
x1 = np.tile(np.arange(1, 5), (3, 1)).T.flatten()
x2 = np.tile(np.arange(1, 4), (4, 1)).flatten()
model_dict = {'x1':x1, 'x2':x2, 'y':y}
model = sm.formula.ols('y~C(x1)+C(x2)', model_dict).fit()
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(x1),3.0,157.590000,52.530000,0.430586,0.738747
C(x2),2.0,223.846667,111.923333,0.917429,0.449118
Residual,6.0,731.980000,121.996667,NaN,NaN


由 $p$ 值可知各种燃料和推进器之间的差异对火箭射程无显著影响

#### 3. 关于交互效应的双因素方差分析
当 $H_{03}$ 成立时，
$$
F_{AB}=\dfrac{\dfrac{S_{AB}}{(r-1)(s-1)}}{\dfrac{S_E}{rs(t-1)}}\sim F((r-1)(s-1), rs(t-1))
$$
检验规则为
$$
\begin{align*}
&F_A>F_\alpha(s-1, rs(t-1)){\text 时拒绝{\mathit H_{01}}}\\
&F_B>F_\alpha(r-1, rs(t-1)){\text 时拒绝{\mathit H_{02}}}\\
&F_{AB}>F_\alpha((r-1)(s-1), rs(t-1)){\text 时拒绝{\mathit H_{03}}，认为交互作用显著}
\end{align*}
$$
双因素方差分析表
| 方差来源 | 离差平方和 |    自由度    |         均方          |                      $F$ 值                       |
| :------: | :--------: | :----------: | :-------------------: | :-----------------------------------------------: |
| 因素 $A$ |   $S_A$    |    $s-1$     |      $S_A/(s-1)$      |      $F_A=\dfrac{S_A/(s-1)}{S_E/[rs(t-1)]}$       |
| 因素 $B$ |   $S_B$    |    $r-1$     |      $S_B/(r-1)$      |      $F_B=\dfrac{S_B/(r-1)}{S_E/[rs(t-1)]}$       |
| 交互效应 |  $S_{AB}$  | $(r-1)(s-1)$ | $S_{AB}/[(r-1)(s-1)]$ | $F_{AB}=\dfrac{S_{AB}/(r-1)(s-1)}{S_E/[rs(t-1)]}$ |
|   误差   |   $S_E$    |  $rs(t-1)$   |    $S_E/[rs(t-1)]$    |                                                   |
|   总和   |   $S_T$    |   $rst-1$    |                       |                                                   |

In [5]:
data = np.loadtxt('../../09第9章  数据的描述性统计方法/data9_21.txt')
x1 = np.tile(np.arange(1, 5), (6, 1)).T
x2 = np.tile(np.array([1,1,2,2,3,3]), (4, 1))
model_dic = {'x1': x1.flatten(), 'x2': x2.flatten(), 'y': data.flatten()}
model = sm.formula.ols('y~C(x1)*C(x2)', model_dic).fit()
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(x1),3.0,261.675000,87.225000,4.417388,0.025969
C(x2),2.0,370.980833,185.490417,9.393902,0.003506
C(x1):C(x2),6.0,1768.692500,294.782083,14.928825,0.000062
Residual,12.0,236.950000,19.745833,NaN,NaN


因为 $p$ 值均小于 $0.05$，故可拒绝均值相等假设，即认为不同燃料（因素 $A$）、不同推进器（因素 $B$）下的射程有显著差异，交互作用也是显著的